In [ ]:
!pip install pennylane

**Introduction to Applications of Quantum Computing to Quantum Chemistry**

**Problem 2: challenge**

**Item (b)**

In this notebook, we implement a variational circuit to estimate the ground state energy of the transverse Ising hamiltonian. The first cell loads the necessary libraries, the second one defines a function to generate the Ising hamiltonian. Finally, the third cell constructs the variational circuit and the optimization procedure.

A few auxiliary lines are commented, but can help to illustrate the proposed solution. For instance, visualizing the circuit and the estimated values for the cost function.

In [4]:
import pennylane as qml
from pennylane import numpy as pnp
import matplotlib.pyplot as plt

In [5]:
# define the hamiltonian for the Ising model

def ising_hamiltonian(h, num_qubits):
  hamiltonian = - qml.PauliZ(0) @ qml.PauliZ(num_qubits - 1) -sum(
      qml.PauliZ(i) @ qml.PauliZ(i+1) for i in range(num_qubits-1)) - h*sum(
          qml.PauliX(i) for i in range(num_qubits))

  return hamiltonian

# # check if the hamiltonian has the right form for 3 qubits and h = 1.2:
# print(ising_hamiltonian(1.2, 3))

In [8]:
# set the number of qubits and the strength of the magnetic field
num_qubits = 4
h = 1.5

# set Pennylane device and the variational circuit
dev = qml.device("lightning.qubit", wires = num_qubits)

@qml.qnode(dev)
def circuit(params):
  # initialize the qubits in a uniform superposition of |0> and |1>
  for i in range(num_qubits):
    qml.Hadamard(wires = i)

  # apply rotations to all qubits. The first half of the 'params' array contains
  # the Z rotation angles, the second half contains the Y rotation angles. No X
  # rotations are considered, given the initial state is an eigenstate of X.
  for i in range(num_qubits):
    qml.RZ(params[i], wires = i)
    qml.RY(params[i + num_qubits], wires = i)

  # return the expectation value of the hamiltonian
  return qml.expval(ising_hamiltonian(h, num_qubits))


# define cost function (expected value of hamiltonian)
def cost(params):
    estimated_energy = circuit(params)
    return estimated_energy


# Choose the optimizer and a step size
opt = qml.GradientDescentOptimizer(stepsize = 0.05)


# create array of parameters
params = pnp.array(pnp.zeros(2*num_qubits), requires_grad = True)


# set initial guess (angles to be optimized)
for i in range(2*num_qubits):
  params[i] = 0.1

## uncomment to visualize the circuit!
# print(qml.draw(circuit)(params))

# open lists for plotting the results
plot_x = []
plot_y  = []

# loop over a number of steps
for i in range(200):
    params, previous_cost = opt.step_and_cost(cost,params)

    if i % 10 == 0:
        # print the results for steps multiples of 10
        # print(f'Step: {i}, Cost: {cost(params)}')
        plot_x.append(i)
        plot_y.append(cost(params))


# print the ground state energy after the optimization
print(f'Ground state energy: {cost(params)}')

# plot the results
plt.plot(plot_x,plot_y, 'o-', label = r'$\langle H \rangle $')
plt.xlabel('Iterations')
plt.ylabel('Cost function')
plt.title(f'Transverse Ising model for {num_qubits} qubits')
plt.legend(loc = 'best')

# horizontal line to visualize convergence
plt.axhline(y = min(plot_y), color = 'black', linestyle = ':')
plt.show()

Ground state energy: -6.249995260552641
